# Analiza koszykowa

## Cel analizy oraz rekomendowana metoda analizy:

Zaprezentowanie przykładowego rozwiązania wspierającego proces rekomendacji produktów, bazujące na analizie powiazania pomiędzy wcześniejszymi zakupami dokonanymi przez klientów.

## Import wykorzystanych bibliotek:

In [2]:
import pandas as pd 
import numpy as np 
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
zakupy=pd.read_csv("dane.csv")
zakupy.head()

,ID_produkt,Ilosc,Data,ID_klient,Kraj
0,A11467,6,2019-11-29 08:26:00,638541.0,United Kingdom
1,A11468,6,2019-11-29 08:26:00,638541.0,United Kingdom
2,A11469,8,2019-11-29 08:26:00,638541.0,United Kingdom
3,A11470,6,2019-11-29 08:26:00,638541.0,United Kingdom
4,A11471,6,2019-11-29 08:26:00,638541.0,United Kingdom


## Objaśnienie poszczególnych atrybutów zaczerpnięte z treści zadania:
- ID_produkt – unikatowe ID produktu zakupionego przez klienta w danej transakcji
- Ilosc – ilość zakupionych produktów w danej transakcji
- Data – data i godzina transakcji
- ID_klient – unikatowe ID klienta
- Kraj - kraj, z którego zostało złożone zamówienie

In [4]:
zakupy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532429 entries, 0 to 532428
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID_produkt  532429 non-null  object 
 1   Ilosc       532429 non-null  int64  
 2   Data        532429 non-null  object 
 3   ID_klient   397773 non-null  float64
 4   Kraj        532429 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 20.3+ MB


## Transformacja danych na potrzeby utworzenia koszyka zakupowego:
Otrzymane dane nie posiadały kolumny zawierającej identyfikator koszyka zakupowego danego klienta, więc przystąpiono do utworzenia kolumny "Koszyk", która zostanie wykorzystana do analizy korelacji pomiędzy poszczególnymi produktami.
Ponieważ seria "Data" zawiera czas zawarcia danej transakcji co do sekundy, posłużono się nią do rozdzielenia poszczególnych "paragonów".  

In [5]:
zakupy.groupby('Data')
Data = zakupy['Data']

In [6]:
Koszyk = []
loop_range = Data.size
koszyk=1000
for i in range (loop_range):
    if (Data.values[i] != Data.values[i-1]):
        koszyk += 1
        Koszyk.append(koszyk)
    else: 
        Koszyk.append(koszyk)  
Series_Koszyk = pd.Series(Koszyk) 
zakupy['Koszyk'] = Series_Koszyk 

Poszczególny indeks koszyka odpowiada dacie rejestracji transakcji, każdy kolejny otrzymał indeks większy o 1. 

In [7]:
zakupy.head()

,ID_produkt,Ilosc,Data,ID_klient,Kraj,Koszyk
0,A11467,6,2019-11-29 08:26:00,638541.0,United Kingdom,1001
1,A11468,6,2019-11-29 08:26:00,638541.0,United Kingdom,1001
2,A11469,8,2019-11-29 08:26:00,638541.0,United Kingdom,1001
3,A11470,6,2019-11-29 08:26:00,638541.0,United Kingdom,1001
4,A11471,6,2019-11-29 08:26:00,638541.0,United Kingdom,1001


Utworzono tabelę przestawną, w której poszczególne wiersze odpowiadają 1 transakcji. 

In [8]:
zamówienia = zakupy.pivot_table(values='Ilosc', index='Koszyk', columns='ID_produkt', fill_value=0)

In [9]:
zamówienia.head()

ID_produkt,A11467,A11468,A11469,A11470,A11471,A11472,A11473,A11474,A11475,A11476,...,A15504,A15505,A15506,A15507,A15508,A15509,A15510,A15511,A15512,A15513
Koszyk,,,,,,,,,,,,,,,,,,,,,
1001,6.0,6,8.0,6.0,6.0,2.0,6,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
1002,0.0,0,0.0,0.0,0.0,0.0,0,6.0,6.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
1003,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,32.0,...,0,0,0,0,0.0,0.0,0,0,0,0
1004,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
1005,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0


Ponieważ z perspektywy analizy asocjacyjnej ważniejszy jest fakt wystąpienia zakupu danego produktu w relacji do innego, niż jego ilość, tabelę przekonwertowano w następujący sposób: if x <= 0: then 0,     if x >= 1: then 1. Umożliwi to wykorzystanie algorytmu apriori z pakietu mlxtend.

In [10]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
zamówienia_01 = zamówienia.applymap(encode_units)
zamówienia_01

ID_produkt,A11467,A11468,A11469,A11470,A11471,A11472,A11473,A11474,A11475,A11476,...,A15504,A15505,A15506,A15507,A15508,A15509,A15510,A15511,A15512,A15513
Koszyk,,,,,,,,,,,,,,,,,,,,,
1001,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1002,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1003,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20985,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Sprawdzono częstotliwość z jaką występują kombinacje pomiędzy produktami - support(produktA, produktB) = (liczba transakcji gdzie produktA, produktB znajdują się w jednym koszyku)/(wszystkie transakcje)
W tak dużym zbiorze danych spodziewano się progu min_support na niskim poziomie, dobierano go doświadczalnie od 0.6-0.02. 

In [ ]:
kombinacje = apriori(zamówienia_01, min_support=0.02, use_colnames=True)

In [ ]:
reguły = association_rules(kombinacje, metric="lift")

In [ ]:
reguły.head(10)

Powyżej wyświetlono 10 pierwszych pozycji, dla których lift > 1 – interpetować można to jako pozytywna zależność pomiędzy produktami i sądzę, że mogą one posłużyć jako propozycje w przypadku, gdy w koszyku znajduje się produkt z tabeli antecedents.

## Wyjaśnienie działania algorytmu 

W celu zobrazowania zasady działania algorytmu posłużę się klasycznym przykładem cross-sellingu jakim jest: "jeżeli mleko to również chleb". Analiza powiązań produktowych na podstawie danych historycznych nazywana jest analizą koszykową lub asocjacyjną. Polega na proponowaniu kupującym produktów, które kupili inni kupujący, analizując na bieżąco koszyk zakupowy.

### Poszczególne etapy:
- zebranie danych historycznych zawierających zamówienia
- analiza każdego koszyka pod kątem współwystępowania produktów (tworzymy tabele, w której każdy wiersz odpowiada jednej transakcji, natomiast kolumna danemu produktowi)
- określamy miary asocjacyjne support - czyli informacja o tym jak często zdarza się wspólny zakup dwóch produktów. Confidence - czy zakup dwóch produktów to przypadek oraz lift, ile razy częściej klient dokona zakupu obu produktów, niż gdyby były statystycznie niezależne. 

### Wzory:
- support(produktA, produktB) = (liczba transakcji gdzie produktA, produktB znajdują się w jednym koszyku)/(wszystkie transakcje)
- confidence = support(produktA, produktB)/support(produktA)
- lift (produktA, produktB) = support(produktA, produktB)/(support(produktA)*support(produktB)
- w praktyce najczęściej do oceny, który produkt zaproponować wybiera się miarę confidence, ponieważ warto wyeliminować czynnik losowości